In [1]:
import pandas as pd
import datetime
import os
import re
import subprocess
from IPython.display import display
import duckdb

# Configuracion

In [2]:
# Ruta al archivo .accdb
# Fixed base path
base_path = '/Users/jordiariassantaella/Python_Projects/Alpha_Espai/POC/SUMAS_Y_SALDOS/'

# Modified paths
db_path = base_path
db_path_INTERMEDIATE = base_path + 'INTERMEDIATE/'
db_path_output = base_path + 'FINAL_RESULT/'

file = "2322024.accdb"  # Cambiar el nombre del archivo según sea necesario
#file = "2302023.accdb"  # Cambiar el nombre del archivo según sea necesario

access_file_path = os.path.join(db_path, file)
file_path = os.path.join(db_path_INTERMEDIATE, file)
file_without_extension = os.path.splitext(file_path)[0]

# Nombre de la tabla que deseas leer
table_name = 'F_APU'  # Cambiar el nombre de la tabla según sea necesario

# Obtener el mes actual
current_month = datetime.datetime.now().month

####
# Parsea nombre fichero para extraer codigo de Franquiciado y año de los datos

# Extraer el código de Franquiciado y el año
last_dot_index = file_path.rfind('.')

if last_dot_index != -1:
    codigo_Franquiciado = file_path[last_dot_index - 7:last_dot_index - 4]
    year = file_path[last_dot_index - 4:last_dot_index]
    print(f"Código de Franquiciado: {codigo_Franquiciado}")
    print(f"Año: {year}")
else:
    print("No se encontró un punto en el nombre del archivo.")

# Formatear el nombre del archivo con el mes actual
output_path = f'{file_without_extension}_{current_month}.csv'
print(f"Ruta de salida: {output_path}")

# Formatear el nombre del fichero en crudo
file_name_without_extension = os.path.splitext(file)[0]
raw_file = f'{db_path_INTERMEDIATE}raw_{file_name_without_extension}_{current_month}.csv'

Código de Franquiciado: 232
Año: 2024
Ruta de salida: /Users/jordiariassantaella/Python_Projects/Alpha_Espai/POC/SUMAS_Y_SALDOS/INTERMEDIATE/2322024_12.csv


# Conecta con Database Access y extrae tabla F_APU en un df

In [3]:
# Comando para listar las tablas en la base de datos
list_tables_cmd = f'mdb-tables -1 {access_file_path}'
tables = subprocess.check_output(list_tables_cmd, shell=True).decode().split()

# Verificar si la tabla existe
if table_name in tables:
    # Comando para exportar la tabla a un archivo CSV
    export_cmd = f'mdb-export {access_file_path} {table_name}'
    df = pd.read_csv(subprocess.Popen(export_cmd, shell=True, stdout=subprocess.PIPE).stdout)

    # Mostrar el DataFrame utilizando display
    #display(df)
else:
    print(f"La tabla '{table_name}' no existe en la base de datos.")

File not found
Couldn't open database.


CalledProcessError: Command 'mdb-tables -1 /Users/jordiariassantaella/Python_Projects/Alpha_Espai/POC/SUMAS_Y_SALDOS/2322024.accdb' returned non-zero exit status 1.

# Monta tabla con los datos relevantes

In [24]:
# Extrae columnas necesarias
query = 'SELECT FECAPU, CUEAPU, CONAPU, IMPAPU, IMEAPU, "D-HAPU", DEPAPU FROM df'
result = duckdb.sql(query)

# Convert to DataFrame
result_df = result.fetchdf()

# Crea nuevas columnas para nombre de fichero
result_df['Mes'] = current_month
result_df['Año'] = year
result_df['Franquiciado'] = codigo_Franquiciado

# Mostrar el DataFrame utilizando display
# display(result_df)

# Elimina filas con codigo DEPAPU igual a 0 (no asignados a una tienda) y ordena por DEPAPAU

In [25]:
# Combine filtering and sorting into a single SQL query
query = """
SELECT *
FROM result_df
WHERE DEPAPU != 0
ORDER BY CUEAPU;
"""

# Execute the query and get the result as a DataFrame
filtered_sorted_df = duckdb.sql(query).df()

# Display the filtered DataFrame
# display(filtered_sorted_df)

# Salva tabla en crudo sin agregar valores con fromato raw_XXX20XX.csv

In [26]:
# Guardar el DataFrame result en un archivo CSV
#file_name_without_extension = os.path.splitext(file)[0]
#raw_file = f'{db_path}raw_{file_name_without_extension}_{current_month}.csv'

filtered_sorted_df.to_csv(raw_file, index=False)

# Sumar IMEAPU por coincidencia de codigo CUEAPU cuando sean iguales valores de D-HAPU y DEPAPU


In [27]:
query = """
SELECT
    CUEAPU,
    "D-HAPU",
    DEPAPU,
    Año,
    Mes,
    Franquiciado,
    SUM(IMEAPU) as total_IMEAPU,
    COUNT(*) as cantidad_registros
FROM filtered_sorted_df
GROUP BY CUEAPU, "D-HAPU", DEPAPU, Año, Mes, Franquiciado;
"""

final_df = duckdb.sql(query).df()
# display(final_df)

# Incorporar info Franquiciado por codigo DEAPU proveniente de fichero Masterfile_CODIGOS.csv

In [28]:
# First, read the CSV file into a table in DuckDB
query_import = f"""
CREATE OR REPLACE TABLE codigos AS
SELECT * FROM read_csv_auto('{db_path}/Masterfile_CODIGOS.csv');
"""
duckdb.sql(query_import)

# Now, perform the JOIN and create the final table
query_join = """
CREATE OR REPLACE TABLE final_table AS
SELECT
    f.*,  -- all fields from final_df
    c.SOCIEDAD,
    c.CIF,
    c.ATICA,
    c.Provincia,
    c.TIENDA,
    c."Nº Tiendas",
    c.sociedad_2,

FROM final_df f
LEFT JOIN codigos c ON f.DEPAPU = c.EST;  -- changed DEPAPU to EST
"""
duckdb.sql(query_join)

# Visualize the result
final_df = duckdb.sql("SELECT * FROM final_table").df()
# display(final_df)

# Salvar fichero Franquiciado (puede incluir más de una tienda)

In [29]:
# Guardar el DataFrame result en un archivo CSV
#display(final_df)
display (output_path)
final_df.to_csv(output_path, index=False)

'/Users/jordiariassantaella/Python_Projects/Alpha_Espai/POC/SUMAS_Y_SALDOS/INTERMEDIATE/2322024_10.csv'

# Desagrega el fichero final por tienda (columna DEPAPU) y registra en directorio FINAL_RESULT con nombre: **Tienda_DEPAPU_año_mes.csv**

In [30]:
# Function to clean the TIENDA value
def clean_tienda_value(tienda_value):
    # Ensure the value is a string
    if not isinstance(tienda_value, str):
        return str(tienda_value)

    # Use regex to remove any number at the end of the string, optionally preceded by a comma
    cleaned_value = re.sub(r',?\s*\d+(,\s*)?$', '', tienda_value)
    return cleaned_value

# Loop through unique DEPAPU values
for depapu_value in final_df['DEPAPU'].unique():
    # Filter the DataFrame based on the current DEPAPU value
    df_filtered = final_df[final_df['DEPAPU'] == depapu_value]

    # Loop through unique TIENDA values within the filtered DataFrame
    for tienda_value in df_filtered['TIENDA'].unique():
        # Clean the TIENDA value
        cleaned_tienda_value = clean_tienda_value(tienda_value)

        # Filter the DataFrame further based on the current TIENDA value
        df_tienda = df_filtered[df_filtered['TIENDA'] == tienda_value]

        # Check if the DataFrame is not empty
        if not df_tienda.empty:
            # Extract year and month values (assuming you have those columns)
            year = df_tienda['Año'].iloc[0]  # Assuming year is consistent for each TIENDA
            mes = df_tienda['Mes'].iloc[0]  # Assuming month is consistent for each TIENDA

            # Construct the filename
            filename = f"{db_path_output}{cleaned_tienda_value}_{depapu_value}_{year}_{mes}.csv"
            #print (filename)

            # Save the filtered DataFrame to a CSV file
            df_tienda.to_csv(filename, index=False)

# Print a message when the process is done
print("Datos separados en archivos CSV.")

Datos separados en archivos CSV.
